# Batch Effects Graphs Description

This Jupyter Notebook makes all of the PCA plots showing the relationship between histologies and batch. 

This script works best with Jupyter Notebook Code Folding and Collapsable Header Extensions Added. To run this you will need to download the R Kernal. You can do that by typing the following into the command line:

```conda install -c r r-irkernel```

Go to this [link](https://forums.fast.ai/t/useful-jupyter-notebook-tips-plugins-collapsible-sections/17919) to learn how to enable collapsable headers.

To export this file as an HTML with collapsable headers type the following into the command line

```jupyter nbconvert --template=collapsible_headings --to html_ch "graph-generator.ipynb"```

# Library Import & Data Download

#### Import the necessary libararies 

In [ ]:
library(tibble)
library(tidyr)
library(ggplot2)
library(readr)
library(dplyr)
library(BatchQC)
library(forcats)
library(sva)
library(rprojroot)

####  Configure file paths & import functions

In [ ]:
# Configure file paths
root_dir = find_root(has_file("OpenPBTA-analysis.Rproj"))
analysis_dir = file.path(root_dir, "analyses", "batch-effects")
data_dir = file.path(root_dir, "data")
functions = file.path(analysis_dir, "util", "functions.R")

In [ ]:
# Load functions from /util/function.R
source(functions)

#### Download the Data

In [ ]:
# Download all covariate data which will be used to identify batches
covariate_file = file.path(data_dir, "pbta-histologies.tsv")
covariate = read_tsv(covariate_file, col_types = cols(molecular_subtype = "c"))

In [ ]:
# Download gene expression data
dat_rsem_polya_file = file.path(data_dir, "pbta-gene-expression-rsem-tpm.polya.rds")
dat_rsem_polya = readRDS(dat_rsem_polya_file)

dat_rsem_stranded_file = file.path(data_dir, "pbta-gene-expression-rsem-tpm.stranded.rds")
dat_rsem_stranded = readRDS(dat_rsem_stranded_file)

dat_kallisto_stranded_file = file.path(data_dir, "pbta-gene-expression-kallisto.stranded.rds")
dat_kallisto_stranded <- readRDS(dat_kallisto_stranded_file)
dat_kallisto_stranded = dat_kallisto_stranded[,2:ncol(dat_kallisto_stranded)]

dat_kallisto_polya_file = file.path(data_dir, "pbta-gene-expression-kallisto.polya.rds")
dat_kallisto_polya <- readRDS(dat_kallisto_polya_file)

##### Shorten the data files if you don't have a strong enough computer 

***This should not be run for a full analysis!***

In [ ]:
# Shorten Data Files
dat_rsem_polya = shorten(dat_rsem_polya)
dat_rsem_stranded = shorten(dat_rsem_stranded)
dat_kallisto_stranded = shorten(dat_kallisto_stranded)
dat_kallisto_polya = shorten(dat_kallisto_polya)

#### Group the Kallisto Data

In [ ]:
# Summarize the kallisto data on the gene level in order to join with batch
dat_kallisto_polya = grouper(dat_kallisto_polya)
dat_kallisto_stranded = grouper(dat_kallisto_stranded)

# Sequence Center Batch effects

### Part 1: ***Rsem-tpm Files***

#### Part 1.1: Rsem-Polya

##### Prepare Data

In [ ]:
id_batch_histology = make_id_batch_histology(covariate, "seq_center")

In [ ]:
my_rsem_polya_plots = make_histology_pca_plots(dat_rsem_polya, 
                                               id_batch_histology, 
                                               gene_id = dat_rsem_polya$gene_id, 
                                               report_name = "rsem_polya_sequence",
                                               "pbta-gene-expression-rsem-tpm-combat-seq-center.polya.rds")

##### Graph before adjusting for batch effects

In [ ]:
my_rsem_polya_plots[1]

- File = "pbta-gene-expression-rsem-tpm.polya.rds"
- Batch = sequence center
- Conclusion: **There are batch effects**. On a small scale analysis, you can see that batch 1 groups around the **top left corner** while batch 2 groups in the **bottom left corner**.

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_rsem_polya_plots[2]

- Batch effects are now no longer present because batches one and two seem to be evenly distributed


#### Part 1.2:  Rsem-Stranded

##### Prepare Data 

In [ ]:
my_rsem_stranded_plots = make_histology_pca_plots(dat_rsem_stranded, id_batch_histology, gene_id = dat_rsem_stranded$gene_id, report_name = "rsem_stranded_sequence", "pbta-gene-expression-rsem-tpm-combat-seq-center.stranded.rds")

##### Graph before adjusting for batch effects

In [ ]:
my_rsem_stranded_plots[1]

- File = "pbta-gene-expression-rsem-tpm.stranded.rds"
- Batch = sequence center
- Conclusion: **Cannot determine** from this image if there are batch effects. BatchQC is required (see script for batch-sequence-effects.R). Visually it appears that there are none.

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_rsem_stranded_plots[2]

- No significant changes made to histologies, Batch one doesn't focus as much on the top right quadrant. This batch correction may not be necessary though.

### Part 2: ***Kallisto Files***

#### Part 2.1: Kallisto Polya

##### Prepare Data

In [ ]:
my_kallisto_polya_plots = make_histology_pca_plots(dat_kallisto_polya, id_batch_histology, gene_id = dat_kallisto_polya$gene_id, report_name = "kallisto_polya_sequence", "pbta-gene-expression-kallisto-combat-seq-center.polya.rds")

##### Graph before adjusting for batch effects

In [ ]:
my_kallisto_polya_plots[1]

- File = "pbta-gene-expression-kallisto.polya.rds"
- Batch = sequence center
- Conclusion: **Cannot determine** from this image if there are batch effects. BatchQC is required (see script for batch-sequence-effects.R). Visually it appears that there are none.

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_kallisto_polya_plots[2]

- Batch correction seems unecessary for this file. The changes doesn't seem particularly useful.

#### Part 2.2: Kallisto Stranded

##### Prepare Data

In [ ]:
my_kallisto_standed_plots = make_histology_pca_plots(dat_kallisto_stranded, id_batch_histology, gene_id = dat_kallisto_stranded$gene_id, report_name = "kallisto_stranded_sequence", "pbta-gene-expression-kallisto-combat-seq-center.stranded.rds")

##### Graph before adjusting for batch effects

In [ ]:

my_kallisto_standed_plots[1]

- File = "pbta-gene-expression-kallisto.stranded.rds"
- Batch = sequence center
- Conlusion: There appears to be batch effects. Batch 2 tends to group around the bottom of the graph

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_kallisto_standed_plots[2]

- Batch Effects no longer present because batches one and two seem to be evenly distributed

# Cohort Batch effects

### Part 1: ***Rsem-tpm Files***

#### Part 1.1: Rsem-Polya

##### Prepare Data

In [ ]:
id_batch_histology = make_id_batch_histology(covariate, "cohort")

In [ ]:
my_rsem_polya_plots = make_histology_pca_plots(dat_rsem_polya, 
                                               id_batch_histology, 
                                               gene_id = dat_rsem_polya$gene_id, 
                                               report_name = "rsem_polya_cohort",
                                               "pbta-gene-expression-rsem-tpm-combat-cohort.polya.rds")

##### Graph before adjusting for batch effects

In [ ]:
my_rsem_polya_plots[1]

- File = "pbta-gene-expression-rsem-tpm.polya.rds"
- Batch = sequence center
- Conclusion: There are batch effects. You can see that batch 1 groups around the top left corner while batch two groups in the bottom left corner
- ***NOTE***: For this gene expression file, there are only 59 patients. As such, the batches cohort and sequence center overlap perfectly. Therefore, one cannot remove the batch effects of one without removing the batch effects of the other

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_rsem_polya_plots[2]

- Batch Effects no longer present


#### Part 1.2:  Rsem-Stranded

##### Prepare Data (there is only one batch so this shouldn't work)

In [ ]:
my_rsem_stranded_plots_temp = make_histology_pca_plots(dat_rsem_stranded, id_batch_histology, gene_id = dat_rsem_stranded$gene_id, report_name = "rsem_stranded_cohort", "pbta-gene-expression-rsem-tpm-combat-cohort.stranded.rds")

***NOTE***: because there is only one cohort batch in the file above, an error message should have popped up showing that ComBat failed to run

##### Graph before adjusting for batch effects

In [ ]:
my_rsem_stranded_plots_temp[1]

- File = "pbta-gene-expression-rsem-tpm.stranded.rds"
- Batch = Cohort
- Conclusion: No batch effects.

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_rsem_stranded_plots_temp[2]

- No graph to show

### Part 2: ***Kallisto Files***

#### Part 2.1: Kallisto Polya

##### Prepare Data

In [ ]:
my_kallisto_polya_plots = make_histology_pca_plots(dat_kallisto_polya, id_batch_histology, gene_id = dat_kallisto_polya$gene_id, report_name = "kallisto_polya_cohort", "pbta-gene-expression-kallisto-combat-cohort.polya.rds")

##### Graph before adjusting for batch effects

In [ ]:
my_kallisto_polya_plots[1]

- File = "pbta-gene-expression-kallisto.polya.rds"
- Batch = cohort
- Conclusion: **Cannot determine** from this image if there are batch effects. BatchQC is required (see script for batch-sequence-effects.R). Visually it appears that there are none.
- ***NOTE***: Batches cohort and sequence center completely overlap

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_kallisto_polya_plots[2]

- Batch correction seems unecessary for this file. The changes doesn't seem particularly useful.

#### Part 2.2: Kallisto Stranded

##### Prepare Data

In [ ]:
my_kallisto_standed_plots = make_histology_pca_plots(dat_kallisto_stranded, id_batch_histology, gene_id = dat_kallisto_stranded$gene_id, report_name = "kallisto_stranded_cohort", "pbta-gene-expression-kallisto-combat-cohort.stranded.rds")

##### Graph before adjusting for batch effects

In [ ]:

my_kallisto_standed_plots[1]

- Only one Batch

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_kallisto_standed_plots[2]

- There was only one batch so it doesn't make sense to adjust for batch effects

# Method (polyA vs stranded) Batch effects

### Part 1: ***Rsem Files***

##### Prepare Data

In [ ]:
id_histology = cbind("Kids_First_Biospecimen_ID" = covariate$Kids_First_Biospecimen_ID, "histology" = covariate$short_histology)
colnames(id_histology) <- c("Kids_First_Biospecimen_ID", "histology")
id_histology = as_tibble(id_histology)

In [ ]:
my_rsem_plots = make_histology_pca_plots_methods(dat_rsem_polya, dat_rsem_stranded, id_histology, report_name = "rsem_method")

##### Graph before adjusting for batch effects

In [ ]:
my_rsem_plots[1]

- Files = "pbta-gene-expression-rsem-tpm.polya.rds", "pbta-gene-expression-rsem-tpm.stranded.rds"
- Batch = sequence center
- Conclusion: There are batch effects. You can see that batch 1 groups around the top left corner while batch two groups in the bottom left corner

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_rsem_plots[2]

- Batch Effects no longer present
- You can still see the difference in histologies though. However, HGAT and LGAT show more overlap than they did previously


### Part 2: ***Stranded Files***

#### Part 2.1: Kallisto Polya

##### Prepare Data

In [ ]:
my_kallisto_plots = make_histology_pca_plots_methods(dat_kallisto_polya, dat_kallisto_stranded, id_histology, report_name = "kallisto_method")

##### Graph before adjusting for batch effects

In [ ]:
my_kallisto_plots[1]

- Files = "pbta-gene-expression-kallisto-tpm.polya.rds", "pbta-gene-expression-kallisto-tpm.stranded.rds"
- Batch = sequence center
- Conclusion: There are batch effects. You can see that batch 1 groups around the top left corner while batch two groups in the bottom left corner

##### Show batch effects after adjustment via [combat](https://rdrr.io/bioc/sva/man/ComBat.html)

In [ ]:
my_kallisto_plots[2]

- Batch Effects no longer present
- You can still see the difference in histologies though. However, HGAT and LGAT show more overlap than they did previously

